In [39]:
# importa as bibliotecas necessárias
import PyPDF2
import re
import pandas as pd

In [3]:

# Abre o arquivo pdf 
path = r'G:\Drives compartilhados\Pontte Crédito\0_HOME EQUITY\0_Analises\FAUSTO MARTINS ROBOREDO\KIT QI\HE_Contrato_Fausto_vf.pdf'
pdf_file = open(path, 'rb')


In [4]:
#Faz a leitura usando a biblioteca
read_pdf = PyPDF2.PdfFileReader(pdf_file)

# pega o numero de páginas
number_of_pages = read_pdf.getNumPages()

#Extriar Texto Página 1 a 5
text=''
for i in range(0,4):
    #Ler Página PDF
    pageObj = read_pdf.getPage(i)
    #Extrair Texto
    text=text+pageObj.extractText()

#Tratar Texto (Remover Quebra de Linhas)
text = re.sub('\r', '', text) 
text = re.sub('\n', '', text)

#Mostrar Texto Tratado
#print(text)

In [22]:
textoBruto = "VALOR DO EMPRÉSTIMO: R$"
valorBruto = text.find(textoBruto,0)
virgula = text.find(",",valorBruto)

#Criar Intervalo de Extração
inicioBruto = len(textoBruto) + valorBruto # <-- Paga o Intervalo que Inicia os números do valor bruto.
finalBruto = virgula+3 # +3 <-- Para incluir virgula + 2 casas decimais, pegando o final do valor bruto.

#Extrair Conteúdo
valorBruto = text[inicioBruto:finalBruto]
float(valorBruto.replace(".","").replace(",","."))


377964.46

In [127]:
listaDePara = {'valorTotal':'VALOR DO EMPRÉSTIMO: R$','tabela': 'SISTEMA DE AMORTIZAÇÃO:','iofCorrigido': 'IOF: R$',
                'tag': 'TARIFA DE ANÁLISE DE GARANTIAS: R$','registro': 'DESPESAS DE REGISTRO: R$',
                'valorLiquido': 'VALOR LÍQUIDO DO EMPRÉSTIMO (A-J-K-L-M-N): R$','prazoMes': 'PRAZO DE AMORTIZAÇÃO:',
                'taxa': 'H.1. NOMINAL:', 'primeiraParcela':'VALOR DA PARCELA MENSAL (AMORTIZAÇÃO E JUROS), NESTA DATA: R$',
                'valorImóvel':'LEILÃO PÚBLICO   IMÓVEL: R$ ','cet':'CUSTO EFETIVO TOTAL (CET):','prazoContrato': 'N.º DE PRESTAÇÕES:',
                'ultimaParcela':'DATA DO TÉRMINO DO PRAZO CONTRATUAL:',
                'dataContrato': 'DATA DE DESEMBOLSO:'
                }

#len(listaHE) # <--- Qtd de Itens na Lista
listaKey = []
listaValues = []

for key, value in listaDePara.items():
    inicioFrase = text.find(value,0)
    finalFrase = inicioFrase + len(value) + 1
    proximoEspaco = text.find(" ", finalFrase)
    valorExtraido = text[finalFrase:proximoEspaco]

    #Ajustar Valores Númericos
    if '.' in valorExtraido:
        valorExtraido = valorExtraido.replace(".", "")
        valorExtraido = valorExtraido.replace(",", ".")
    
    #Ajustar Valores Percentuais
    if '%' in valorExtraido:
        valorExtraido = valorExtraido.replace(",", ".")
        valorExtraido = valorExtraido.replace("%", "")
        valorExtraido = round(float(valorExtraido)/100,4)
    
    listaKey.append(key)
    listaValues.append(valorExtraido)

    #print(key + ": ", valorExtraido)

#Criar Dicionario das duas Listas
dict_keyValue = dict(zip(listaKey,listaValues))

#Criar DF a partir do Dicionario
df = pd.DataFrame([dict_keyValue])

#Criar Colunas Dos Campos que estão faltando
df.insert(len(listaKey),"carencia", [''])
df.insert(len(listaKey)+1,"diaPagto", [''])

#Inserir Valores Nas Colunas
df['carencia'] = df['prazoMes'].astype(float) - df['prazoContrato'].astype(float) -1
df['diaPagto'] = pd.DatetimeIndex(df['ultimaParcela']).day

df


,valorTotal,tabela,iofCorrigido,tag,registro,valorLiquido,prazoMes,taxa,primeiraParcela,valorImóvel,cet,prazoContrato,ultimaParcela,dataContrato,carencia,diaPagto
0,377964.46,SAC,12640.27,12250.00,3074.19,350000.00,228,0.1255,5675.21,62000.00,0.1497,224,15/07/2040,06/07/2021,3.0,15


In [165]:
import sys
import os

file_name = os.path.basename(sys.path[0])
pathBruto = sys.path[0]

posicaoFinal = len(pathBruto) - len(file_name) - 1

pathTratado = (pathBruto[0:posicaoFinal]).capitalize()
pathTratado


'G:\\drives compartilhados\\pontte\\operações\\automações\\projetos testes\\code de validação test'

In [205]:
#Lógica para localizar arquivo na pasta .PDF <--- Contrato

#Função Que Acha Arquivo
def caminhoArquivo(pathProcurar, palavra1, palavra2, extensao):
    for root, dirs, files in os.walk(pathProcurar):
        for file in files:
            if palavra1 in file or palavra2 in file:
                if extensao in file:
                    pathArquivo = os.path.join(root, file)
                    return pathArquivo


#Palavras Chaves
palavraChave1 = 'Contrato'
palavraChave2 = 'contrato'
palavraChave3 = 'Cálculo'
palavraChave4 = 'Calculo'

#Extensão do Arquvo
xlsx = '.xlsx'
pdf = '.pdf'

arquivoPDF = caminhoArquivo(pathTratado, palavraChave1, palavraChave2, pdf)
arquivoXLSX = caminhoArquivo(pathTratado, palavraChave3, palavraChave4, xlsx)


print("Caminho da planilha:", arquivoXLSX)
print("\nCaminho do PDF:", arquivoPDF)

Caminho da planilha: G:\drives compartilhados\pontte\operações\automações\projetos testes\code de validação test\Cálculo_fluxo.xlsx

Caminho do PDF: G:\drives compartilhados\pontte\operações\automações\projetos testes\code de validação test\Leitor de PDF\HE_Contrato_Fausto_vf.pdf
